# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk

In [2]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 11:39 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [5]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_bucket_sy' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [6]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [7]:
from inverted_index_gcp import *

InvertedIndex.DIR_NAME = "title_index/postings_gcp_title_index"

In [8]:
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix = 'wiki_files/')
for b in blobs:
    if b.name != 'wiki_files/graphframes.sh' and b.name != 'wiki_files/' :
        paths.append(full_path+b.name)


In [9]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

inverted_title = InvertedIndex()
inverted_title.DIR_NAME = InvertedIndex.DIR_NAME

In [10]:
# Count number of wiki pages
N_docs = parquetFile.count()

inverted_title.N = N_docs

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

DIR_NAME = "title_index/postings_gcp_title_index"

NUM_BUCKETS = 124

def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
    A list of (token, (doc_id, tf)) pairs 
    for example: [("Anarchism", (12, 5)), ...]'''

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    token_counter = Counter(x for x in tokens if x not in all_stopwords)
    res = [(token, (id, tf)) for token, tf in token_counter.items()]
    return res

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
    Returns:
    --------
    list of tuples
      A sorted posting list.'''

    return list(sorted(unsorted_pl))

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.'''

    return postings.map(lambda x: (x[0],len(x[1])))

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.'''

    buckets_split = postings.map(lambda x: (token2bucket_id(x[0]),x))
    buckets_group = buckets_split.groupByKey()
    return buckets_group.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))


###### add starts here

def count_doc_len(doc_id, text):
    '''this functen calculates the document len given the dic id and text '''

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    num_tokens = len([x for x in tokens if x not in all_stopwords])
    return (doc_id,num_tokens)



In [12]:
# now lets count the frequency of each word in title
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

In [13]:
# calculate the length of each title and saves values in DL where {key = doc_id: value = doc_length}
len_docs_title = doc_title_pairs.map(lambda x: count_doc_len(x[1], x[0]))
len_docs_title = len_docs_title.collectAsMap()
inverted_title.DL = len_docs_title

In [14]:
#create postings and sort by doc_id
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [15]:
# stores total frequency per term as a counter in "term_total" feild
total_terms_title = postings_title.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_title.term_total = total_terms_title.collectAsMap()

In [16]:
# filtering postings and calculate df for each term. stores it in title_index.df attribute
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()
inverted_title.df = w2df_title_dict

In [17]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

In [18]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=DIR_NAME):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_title[k].extend(v)

#stores it in title_index.posting_locs attribute
inverted_title.posting_locs = super_posting_locs_title

In [19]:
# write the global stats out
inverted_title.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/postings_title_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
\ [1 files][132.9 MiB/132.9 MiB]                                                
Operation completed over 1 objects/132.9 MiB.                                    
